## Parsing Graph Network

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [3]:
def num_nodes_edges():
  def simple_tokenize(line):
     return [int(i) for i in line.split()]
  lines = sc.textFile("p2p-Gnutella08-adj.txt")
  num_nodes = lines.count()
  num_edges = lines.map(lambda x: len(simple_tokenize(x))-1).reduce(lambda x,y:x+y)
  return (num_nodes, num_edges)  
    
def out_counts():
  def simple_tokenize(line):
    loS = [int(i) for i in line.split()]
    return (len(loS)-1, loS[0])
  lines = sc.textFile("p2p-Gnutella08-adj.txt")
  res = lines.map(lambda x: simple_tokenize(x)).countByKey()
  return res
    # """Returns a dictionary where the keys are the outdegrees, and the 
    # values are the number of nodes of the corresponding outdegree """
    # #### Your code for Question 1.2 should go here

def in_counts():
  def simple_tokenize(line):
    loS = [int(i) for i in line.split()]
    return loS[1:]
  lines = sc.textFile("p2p-Gnutella08-adj.txt")
  res = lines.flatMap(lambda x: simple_tokenize(x)).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
  res1 = res.map(lambda x: (x[1], x[0])).countByKey()
  return res1
    # """Returns a dictionary where the keys are the indegrees, and the 
    # values are the number of nodes of the corresponding indegree """
    # #### Your code for Question 1.3 should go here

print(num_nodes_edges(), out_counts(), in_counts())

(6301, 20777) defaultdict(<class 'int'>, {10: 1531, 0: 3836, 9: 372, 1: 294, 3: 16, 7: 9, 2: 28, 5: 107, 8: 44, 6: 10, 4: 28, 13: 2, 14: 2, 18: 2, 25: 1, 12: 3, 29: 1, 19: 1, 47: 1, 34: 1, 24: 1, 22: 1, 28: 1, 11: 1, 17: 4, 46: 1, 48: 1, 31: 1, 41: 1}) defaultdict(<class 'int'>, {6: 227, 59: 1, 1: 2452, 74: 1, 7: 144, 4: 559, 5: 333, 3: 868, 70: 3, 2: 1287, 50: 1, 81: 4, 83: 1, 19: 2, 66: 2, 52: 1, 20: 4, 13: 19, 11: 29, 77: 2, 21: 2, 57: 1, 38: 1, 47: 2, 32: 2, 8: 76, 10: 37, 67: 3, 60: 3, 15: 8, 9: 70, 73: 2, 62: 2, 14: 13, 12: 23, 72: 2, 44: 1, 69: 2, 22: 1, 41: 1, 91: 1, 33: 1, 23: 1, 31: 2, 16: 1, 71: 3, 51: 1, 49: 1, 54: 1, 85: 1, 82: 1, 30: 1, 87: 1, 61: 1, 86: 1, 27: 1, 18: 2, 35: 1, 63: 1, 79: 1, 55: 1, 56: 2, 78: 1, 25: 1})


---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [10]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
  def simple_tokenize(line):
    loS = [int(i) for i in line.split()]
    return (loS[0], loS[1:])
  def pagerank(x):
    if x[1][0] == 0:
      return [(source_node_id, 0)]
    elif x[1][1] == []:
      return [(source_node_id, x[1][0])]
    else: #(100, (1, ["2017"]))
      scores = [(source_node_id, x[1][0] * jump_factor)]
      for i in x[1][1]:
        scores += [(i, x[1][0] * (1 - jump_factor) / len(x[1][1]))]
      return scores
  lines = sc.textFile("p2p-Gnutella08-adj.txt")
  num_nodes = lines.count()
  res = lines.map(lambda x: simple_tokenize(x))
  init_mass = lines.map(lambda x: (simple_tokenize(x)[0], simple_tokenize(x)[0] == source_node_id))
  prev = init_mass
  for i in range(0, num_iterations):
    init_mass = init_mass.join(res).flatMap(lambda x: pagerank(x)).reduceByKey(lambda x, y: x+y)
  init_mass = init_mass.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))
  return init_mass.take(10)
    # """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    # [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # # your solution to Question 2 here
a = personalized_page_rank(100, 10, 0.1)
b = personalized_page_rank(2794, 10, 0.05)
print(sorted(a, key=lambda x: x[1], reverse=True))
print(sorted(b, key=lambda x: x[1], reverse=True))


[(2017, 0.33871974849298303), (100, 0.304745856887131), (2750, 0.020951316194351272), (96, 0.020943898199516482), (3808, 0.02093944720523819), (2794, 0.02093915354613689), (3810, 0.020938613345708006), (1914, 0.020938612231167525), (3807, 0.020938611128676003), (3713, 0.02093860591416992)]
[(2794, 0.36541559072338664), (513, 0.03664950790441285), (2001, 0.03659412767617927), (2292, 0.03646168485025539), (2615, 0.036170134619692956), (1509, 0.03616197848386493), (3215, 0.03614608630353983), (1042, 0.03614599491394114), (4306, 0.03614477608839565), (3907, 0.036144757840226534)]


---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [11]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
  def simple_tokenize(line):
    loS = [int(i) for i in line.split()]
    return (loS[0], loS[1:])
  def pagerank(x):
    if x[1][0] == 0:
      return [(source_node_id, 0)]
    elif x[1][1] == []:
      return [(source_node_id, x[1][0])]
    else: #(100, (1, ["2017"]))
      scores = [(source_node_id, x[1][0] * jump_factor)]
      for i in x[1][1]:
        scores += [(i, x[1][0] * (1 - jump_factor) / len(x[1][1]))]
      return scores
  lines = sc.textFile("p2p-Gnutella08-adj.txt")
  num_nodes = lines.count()
  res = lines.map(lambda x: simple_tokenize(x)).cache()
  init_mass = lines.map(lambda x: (simple_tokenize(x)[0], simple_tokenize(x)[0] == source_node_id))
  prev = init_mass
  max_diff = 1
  while (max_diff >= 0.5 / num_nodes):
    init_mass = init_mass.join(res).flatMap(lambda x: pagerank(x)).reduceByKey(lambda x, y: x+y)
    max_diff = init_mass.join(prev).map(lambda x: abs(x[1][0] - x[1][1])).max()
    prev = init_mass
  init_mass = init_mass.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))
  return init_mass.take(10)
    # """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    # [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # # your solution to Question 3 here
    
a = personalized_page_rank_stopping_criterion(2750, 0.1)
b = personalized_page_rank_stopping_criterion(0, 0.1)
print(sorted(a, key=lambda x: x[1], reverse=True))
print(sorted(b, key=lambda x: x[1], reverse=True))

[(2750, 0.30220394870719763), (7, 0.03464320538320014), (174, 0.033601317145302345), (128, 0.029311063948404763), (856, 0.028554469248364675), (717, 0.028456769423626415), (368, 0.02844554779693093), (424, 0.02815377725062893), (125, 0.027985045265408185), (238, 0.027689556656325633)]
[(0, 0.3405274579137615), (9, 0.032505079338152595), (5, 0.032476399402304686), (7, 0.032362487667159066), (4, 0.03221727201127592), (3, 0.03150003345862), (8, 0.031488430803692735), (10, 0.030872138035464906), (2, 0.030652673309598734), (1, 0.030651790105874478)]


---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

The purpose of cache() calls is to persist an RDD so that it will be kept in memory and can be reused faster later. Giving that it saves time when I resue it multiple times, I am adding it to the first RDD, *res*, in Question 3, since I need to join it in each iteration in the loop. However, I am not adding cache to other RDDs like *init_mass* since it is updated in each iteration. There is no much point caching it.




---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.